#### Load and Prepare Data

In [1]:
import torch
import torchvision
from torchvision import datasets, transforms

print(torch.__version__)

2.0.1+cpu


In [2]:
# Setup the data transformation
transform = transforms.Compose([
    transforms.Resize((128,128)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

data_path = "images"

datasets = datasets.ImageFolder(root = data_path, transform = transform)

In [3]:
# Display class names and index labels for debug
print("Class names: ", datasets.classes)
print("Class index mapping:", datasets.class_to_idx)

Class names:  ['Poop', 'Smile']
Class index mapping: {'Poop': 0, 'Smile': 1}


In [4]:
# Split dataset into training and validation sets
train_size = int(0.8 * len(datasets))
valid_size = len(datasets) - train_size
train_dataset, valid_dataset = torch.utils.data.random_split(datasets, [train_size, valid_size])

# Dataloaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=4, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=4, shuffle = False)


#### Define the Model

In [5]:
import torch.nn as nn
import torch.nn.functional as F

In [6]:
# Set up the neural net
class SimpleCNN(nn.Module):
    def __init__ (self, num_classes):
        super(SimpleCNN, self).__init__()

        self.conv1 = nn.Conv2d(3, 16, 3, padding = 1)
        self.conv2 = nn.Conv2d(16, 32, 3, padding = 1)
        self.fc1 = nn.Linear(32*32*32, 512)
        self.fc2 = nn.Linear(512, num_classes)

    def forward(self, x):
        # first conv, ReLU, pooling
        x = F.max_pool2d(F.relu(self.conv1(x)), 2)
        # second conv, ReLU, pooling
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        # flatten the tensor
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))

        x = self.fc2(x)
        return x
    
num_classes = len(datasets.classes)
model = SimpleCNN(num_classes)

#### Specify Loss Function and Optimizer

In [7]:
lossf = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)


#### Train the Model

In [8]:
epochs = 100

for epoch in range(epochs):
    model.train()
    train_loss = 0.0

    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = lossf(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * inputs.size(0)
    
    train_loss = train_loss / len(train_loader.dataset)
    if(epoch % 10 == 0):
        print(f"Epoch {epoch +1}/{epochs}, Loss: {train_loss: .4f}")




Epoch 1/100, Loss:  1.6006
Epoch 11/100, Loss:  0.6070
Epoch 21/100, Loss:  0.6190
Epoch 31/100, Loss:  0.6108
Epoch 41/100, Loss:  0.6068
Epoch 51/100, Loss:  0.6087
Epoch 61/100, Loss:  0.6053
Epoch 71/100, Loss:  0.6074
Epoch 81/100, Loss:  0.6047
Epoch 91/100, Loss:  0.6039


#### Evaluate the Model

In [10]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in valid_loader:
        output = model(inputs)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        # correct += (predicted == labels).sum().item()

print(predicted.shape)
print(labels.shape)

print(f"Validation Accuracy: {100 * correct / total:.2f}%")

torch.Size([4])
torch.Size([2])
Validation Accuracy: 0.00%
